In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import torch
import torch.nn as nn
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

c:\Users\zhang\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data Generation

In [2]:
NUM_USER = 10000

In [3]:
NUM_Product = 10

In [4]:
treatment_percentage = 0.5

In [5]:
discount = 0.2

In [6]:
user_continuous_feature_multiplier = 1

In [7]:
prod_continuous_feature_multiplier = 1

In [8]:
# Set constants
USER_Cont_FEATURES = 2*user_continuous_feature_multiplier
USER_Dicr_FEATURES = 3

Product_Cont_FEATURES = 3*prod_continuous_feature_multiplier
Product_Dicr_FEATURES = 2
OUTSIDE_OPTION_UTILITY = 0
utilities = torch.zeros(NUM_USER, NUM_Product)

In [9]:
def generate_features(N, C, D):
    continuous_features = np.zeros((N, C))
    for i in range(C):
        continuous_features[:, i] = np.random.uniform(0,1,size=N)
    binary_features = np.random.randint(0, 2, (N, D))
    return np.hstack((continuous_features, binary_features))

In [ ]:
import torch
import torch.nn as nn
import numpy as np

import torch
import torch.nn as nn

class NonparametricUtilityDNN(nn.Module):
    def __init__(self, user_features, product_features):
        super(NonparametricUtilityDNN, self).__init__()
        
        self.original_dim = user_features + product_features + 1
        expanded_dim = (self.original_dim - 1) + 4
        
        self.fc1 = nn.Linear(expanded_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

        nn.init.uniform_(self.fc1.weight, -2, 2)
        nn.init.uniform_(self.fc2.weight, -2, 2)
        nn.init.uniform_(self.fc3.weight, -1, 1)

    def forward(self, x):
        feats = x[:, :-1] 
        

        price = x[:, -1:] 

        p_sq = torch.pow(price, 2)       # Quadratic
        p_quad = torch.pow(price, 4)     # Quartic (Explosive growth)
        p_log = torch.log(torch.abs(price) + 1e-6) # Logarithmic (Decaying growth)

        x_new = torch.cat([feats, price, p_sq, p_quad, p_log], dim=1)
        
        x = torch.tanh(self.fc1(x_new))
        x = torch.tanh(self.fc2(x))
        
        output = self.fc3(x)
        return output



def utility_model_batched_nonparametric(x_user, X_product, price, user_randomization, 
                                        prod_randomization, dnn_model, gumbel_noise, 
                                        discount, batch_size=10):
    
    num_users = x_user.shape[0]
    num_products = X_product.shape[0]
    decisions = torch.zeros(num_users, dtype=torch.long)

    # Convert numpy arrays to tensors if necessary
    if isinstance(user_randomization, np.ndarray):
        user_randomization = torch.from_numpy(user_randomization).to(torch.bool)
    if isinstance(prod_randomization, np.ndarray):
        prod_randomization = torch.from_numpy(prod_randomization).to(torch.bool)
    if isinstance(price, np.ndarray):
        price = torch.from_numpy(price).float()

    # Iterate over users in batches
    for i in range(0, num_users, batch_size):
        batch_end = min(i + batch_size, num_users)
        batch_indices = slice(i, batch_end)
        current_batch_size = batch_end - i


        batch_user_features = x_user[batch_indices].unsqueeze(1).expand(-1, num_products, -1)
        batch_prod_features = X_product.unsqueeze(0).expand(current_batch_size, -1, -1)
        batch_price = price.unsqueeze(0).expand(current_batch_size, -1).unsqueeze(-1) 
        batch_user_treatment = user_randomization[batch_indices].unsqueeze(1).expand(-1, num_products)
        batch_prod_treatment = prod_randomization.unsqueeze(0).expand(current_batch_size, -1)
        is_treated = batch_user_treatment | batch_prod_treatment
        

        batch_adjusted_price = torch.where(
            is_treated.unsqueeze(-1), 
            batch_price * discount, 
            batch_price
        )
        combined_features = torch.cat((batch_user_features, batch_prod_features, batch_adjusted_price), dim=2)
        flat_features = combined_features.view(-1, combined_features.shape[-1])
        utility_preds = dnn_model(flat_features)
        utility_deterministic = utility_preds.view(current_batch_size, num_products)
        
        # Add noise
        batch_utilities = utility_deterministic + gumbel_noise[batch_indices]

        max_utilities, chosen_products = torch.max(batch_utilities, dim=1)

        outside_option = -1 * torch.ones_like(chosen_products, dtype=torch.long)
        decisions[batch_indices] = torch.where(max_utilities > 0, chosen_products, outside_option)

    return decisions

In [11]:
def utility_model(x_user, X_product, price, user_randomization, prod_randomization,pair_utility_model,price_sensitivity_model,gumbel_noise):
    num_users = x_user.shape[0]
    num_products = X_product.shape[0]
    

    price_sensitivities = price_sensitivity_model(x_user)

    for i in range(num_users):

        for j in range(num_products):
            # Determine if the user and product are in the treatment group
            is_user_treated = (user_randomization[i] == 1)
            is_product_treated = (prod_randomization[j] == 1)

            # Adjust price based on the experiment conditions
            adjusted_price = price[j] * discount if is_user_treated or is_product_treated else price[j]
            combined_features = torch.cat((x_user[i], X_product[j]), 0)
            utility_from_dnn = pair_utility_model(combined_features)
            price_effect = price_sensitivities[i] * adjusted_price

            utilities[i, j] = utility_from_dnn - price_effect + gumbel_noise[i,j]

    return utility_from_dnn,price_effect

In [12]:
def make_decision(utilities):
    num_users = utilities.shape[0]
    decisions = torch.zeros(num_users, dtype=torch.long)  
    for i in range(num_users):
        max_utility, chosen_product = torch.max(utilities[i], dim=0)

        # Compare the maximum utility with the outside option (utility = 0)
        if max_utility <= 0:
            decisions[i] = -1 
        else:
            decisions[i] = chosen_product 

    return decisions

In [13]:
def calculate_revenue(decisions, prices):
    total_revenue = 0.0

    # Iterate over each decision and add the corresponding product price to total revenue
    for i, decision in enumerate(decisions):
        if decision != -1:  # Check if the decision is not the outside option
            total_revenue += prices[decision].item()  # Add the price of the chosen product

    return total_revenue

In [14]:
X_user = generate_features(NUM_USER,USER_Cont_FEATURES, USER_Dicr_FEATURES)
X_product = generate_features(NUM_Product, Product_Cont_FEATURES, Product_Dicr_FEATURES)
price = np.random.uniform(0.5 ,1, NUM_Product)

X_user = torch.from_numpy(X_user).float()
X_product = torch.from_numpy(X_product).float()
price = torch.from_numpy(price).float()
gumbel_dist = torch.distributions.Gumbel(0, 1)
gumbel_noise = gumbel_dist.sample((NUM_USER, NUM_Product))

# GTE

## All treated scenario: all products are discounted

In [15]:
user_randomization = np.random.choice([0,1], NUM_USER, p=[1, 0])
prod_randomization = np.random.choice([0,1], NUM_Product, p=[0, 1])

In [16]:
user_dim = X_user.shape[1]
product_dim = X_product.shape[1]
model = NonparametricUtilityDNN(user_dim, product_dim)
decisions_all_treat = utility_model_batched_nonparametric(
    x_user=X_user,
    X_product=X_product,
    price=price,
    user_randomization=user_randomization,
    prod_randomization=prod_randomization,
    dnn_model=model,  
    gumbel_noise=gumbel_noise,
    discount=discount, 
    
    batch_size=10
)

print("Decisions per user (product index or -1 for outside option):\n", decisions_all_treat)

Decisions per user (product index or -1 for outside option):
 tensor([2, 2, 5,  ..., 2, 2, 6])


In [17]:
all_num_unique = torch.unique(decisions_all_treat).numel()
print(all_num_unique)

11


In [18]:
for i in range(-1,10):
    print(torch.sum(decisions_all_treat==i))

tensor(11)
tensor(773)
tensor(145)
tensor(5127)
tensor(1139)
tensor(921)
tensor(179)
tensor(136)
tensor(1043)
tensor(410)
tensor(116)


In [19]:
total_revenue_all_treated = calculate_revenue(decisions_all_treat, price*discount)
print(f"Total revenue from sales when all products are discounted: ${total_revenue_all_treated:.2f}")

Total revenue from sales when all products are discounted: $1654.57


## All control scenario: all products remain the original price

In [20]:
user_randomization = np.random.choice([0,1], NUM_USER, p=[1, 0])
prod_randomization = np.random.choice([0,1], NUM_Product, p=[1, 0])

decisions_all_control = utility_model_batched_nonparametric(
    x_user=X_user,
    X_product=X_product,
    price=price,
    user_randomization=user_randomization,
    prod_randomization=prod_randomization,
    dnn_model=model,  
    gumbel_noise=gumbel_noise,
    discount=discount, 
    
    batch_size=10
)

print("Decisions per user (product index or -1 for outside option):\n", decisions_all_control)
total_revenue_all_control = calculate_revenue(decisions_all_control, price)
print(f"Total Revenue from Sales: ${total_revenue_all_control:.2f}")

Decisions per user (product index or -1 for outside option):
 tensor([8, 3, 8,  ..., 3, 7, 8])
Total Revenue from Sales: $6145.63


In [21]:
revenue_difference = total_revenue_all_treated - total_revenue_all_control
print(f"Revenue Difference (ALLTreated - ALLControl): ${revenue_difference:.2f}")
# print(f"Revenue Relative Difference (ALLTreated - ALLControl)/AllControl: {100*revenue_difference/total_revenue_all_control:.2f}%")

Revenue Difference (ALLTreated - ALLControl): $-4491.07


In [22]:
true = revenue_difference

## product randomization

In [23]:
def calculate_product_revenue(decisions, prices, prod_randomization):
    revenue_treated = 0.0
    revenue_control = 0.0

    # Iterate over each user's decision
    for user_index, decision in enumerate(decisions):
        if decision != -1:  # If the user chose a product
            product_price = prices[decision].item()  # Get the price of the chosen product

            # Check if the product was in the treatment or control group
            if prod_randomization[decision] == 1:
                revenue_treated += product_price
            else:
                revenue_control += product_price

    return revenue_treated, revenue_control

In [24]:
utilities = torch.zeros(NUM_USER, NUM_Product)
user_randomization = np.random.choice([0,1], NUM_USER, p=[1, 0])
prod_randomization = np.random.choice([0,1], NUM_Product, p=[1-treatment_percentage, treatment_percentage])
# prod_randomization = np.random.choice([0,1], NUM_Product, p=[1, ])
decisions_product_randomization  = utility_model_batched_nonparametric(
    x_user=X_user,
    X_product=X_product,
    price=price,
    user_randomization=user_randomization,
    prod_randomization=prod_randomization,
    dnn_model=model,  
    gumbel_noise=gumbel_noise,
    discount=discount, 
    
    batch_size=10
)

In [25]:
revenue_treated, revenue_control = calculate_product_revenue(decisions_product_randomization, price-price*(1-discount)*prod_randomization, prod_randomization)
naive = revenue_treated/treatment_percentage - revenue_control/(1-treatment_percentage)
print(f"Revenue from Treated Products: ${revenue_treated:.2f}")
print(f"Revenue from Control Products: ${revenue_control:.2f}")
print(f"Revenue Difference (Treated - Control) by naive DIM: ${naive:.2f}")
# print(f"Revenue Relative Difference (ALLTreated - ALLControl)/AllControl: {100*revenue_difference/revenue_control:.2f}%")

Revenue from Treated Products: $296.55
Revenue from Control Products: $5186.63
Revenue Difference (Treated - Control) by naive DIM: $-9780.17


## Prepare training and testing data given experiment data

In [26]:
X_user_1, X_user_2, decision_1, decision_2 = train_test_split(
X_user, decisions_product_randomization, test_size=1/2, random_state=3407)


In [27]:
train_set = {
    'features': X_user_1,
    'labels': decision_1
}

test_set = {
    'features': X_user_2,
    'labels': decision_2
}

# Flag to switch between training and test set
use_train_set = False  # Set to False for the test set

# Function to get the current active dataset
def get_active_dataset(use_train):
    return train_set if use_train else test_set
def get_test_dataset(use_train):
    return test_set if use_train else train_set
# Retrieve the current dataset based on the flag
current_dataset = get_active_dataset(use_train_set)
X_user_train = current_dataset['features']
decision_train = current_dataset['labels']
X_user_test = get_test_dataset(use_train_set)['features']
decision_test =  get_test_dataset(use_train_set)['labels']

# use simple MNL structural model

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim

class LinearMNLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(LinearMNLModel, self).__init__()
        # Initialize parameters for user and product features
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0)) 

    def forward(self, x_user, X_product, price, user_randomization, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user and product features to create a [N, M, F] shaped tensor for each
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()


        # Calculate linear utility from features
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Adjust prices based on randomization
        adjusted_price = torch.where(
             prod_randomization.unsqueeze(0),
            price * discount,  
            price
        )

        # Calculate utility from price, properly expanding its dimension
        utility_price = adjusted_price * self.beta_price  # [M]
        utility_price = utility_price.expand(N, M)  # [N, M]

        # Total utility including features and price
        total_utility = utility_user + utility_product + utility_price

        # Incorporate the outside option with utility 0
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)

        return utilities_with_outside





In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_user_train, X_product, price = X_user_train.to(device), X_product.to(device), price.to(device)
if isinstance(user_randomization, np.ndarray):
    user_randomization = torch.from_numpy(user_randomization).to(X_user_train.device).bool()
if isinstance(prod_randomization, np.ndarray):
    prod_randomization = torch.from_numpy(prod_randomization).to(X_user_train.device).bool()

decision_train = decision_train.long().to(device)

In [30]:
model = LinearMNLModel(user_feature_dim=USER_Cont_FEATURES+USER_Dicr_FEATURES,
                       product_feature_dim=Product_Cont_FEATURES+Product_Dicr_FEATURES).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [31]:


num_epochs = 5000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    utilities = model(X_user_train, X_product, price, user_randomization, prod_randomization)
    choice_probabilities = nn.functional.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), decision_train+1])
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')



Epoch 0, Loss: 3.484348773956299
Epoch 1000, Loss: 1.7609167098999023
Epoch 2000, Loss: 1.7473889589309692
Epoch 3000, Loss: 1.7471544742584229
Epoch 4000, Loss: 1.7470839023590088


In [32]:
choice_probabilities

tensor([[-12.8286,  -2.3516,  -3.7096,  ...,  -2.2660,  -3.3253,  -4.1584],
        [-13.4083,  -2.3516,  -3.7096,  ...,  -2.2660,  -3.3253,  -4.1584],
        [-14.9134,  -2.3516,  -3.7096,  ...,  -2.2660,  -3.3253,  -4.1584],
        ...,
        [-17.3158,  -2.3516,  -3.7096,  ...,  -2.2660,  -3.3253,  -4.1584],
        [ -9.7339,  -2.3517,  -3.7096,  ...,  -2.2660,  -3.3254,  -4.1585],
        [-14.9100,  -2.3516,  -3.7096,  ...,  -2.2660,  -3.3253,  -4.1584]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [33]:
beta_price_est = model.beta_price.cpu().detach().numpy()

In [34]:
print(beta_price_est)

-0.9523107


In [35]:
model.beta_user

Parameter containing:
tensor([4.1624, 5.1737, 4.1927, 2.4627, 3.8682], device='cuda:0',
       requires_grad=True)

In [36]:
import torch
import torch.nn.functional as F

In [37]:
all_product_control = np.random.choice([0,1], NUM_Product, p=[1, 0])
all_product_treated = np.random.choice([0,1], NUM_Product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()

X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

utilities = model(X_user_test, X_product, price, user_randomization, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities

# Calculate expected revenue
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

utilities = model(X_user_test, X_product, price, user_randomization, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities

# Calculate expected revenue
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)*discount
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")


Expected Revenue: $3479.85
Expected Revenue: $713.76


In [38]:
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear*2
print(f"Revenue Difference (Treated - Control) by Linear MNL: ${linear:.2f}")
print(f"Absolute Percentage Estimation Error of Linear MNL:  {100*np.abs(linear-revenue_difference)/revenue_difference:.2f}%")


Revenue Difference (Treated - Control) by Linear MNL: $-5532.17
Absolute Percentage Estimation Error of Linear MNL:  -23.18%


# use PDL

In [39]:
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        all_x_other_products.append(other_products)

    # Convert lists to tensor
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
  

    return user_features, product_features, prices, all_x_other_products


In [40]:
X_user_train1, X_user_val, decision_train1,decision_val = train_test_split(X_user_train,decision_train,test_size=0.1,random_state=34)

In [41]:
price = price.to(device)
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount) * prod_randomization))
user_features, product_features, prices, all_x_other_products = prepared_data
user_features.shape, product_features.shape, prices.shape, all_x_other_products.shape

(torch.Size([4500, 5]),
 torch.Size([10, 5]),
 torch.Size([10]),
 torch.Size([10, 45]))

In [42]:
class PDLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(PDLModel, self).__init__()
        # Combined feature dimension includes product features, price, and user features, as well as other products' features and prices
        total_feature_dim = user_feature_dim + 2*product_feature_dim + 1  # +1 for price

        # Single neural network to process the combined features
        self.network = nn.Sequential(
            nn.Linear(total_feature_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5,5),
            nn.ReLU(),
            nn.Linear(5, 1) 
        )
            # Layers to process other products' features (z-j)
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim*(NUM_Product-1), NUM_Product),
            nn.ReLU(),
            nn.Linear(NUM_Product, product_feature_dim)
        )

    def forward(self, x_user, x_product, x_other_products,prices):
        N = x_user.shape[0]
        M = x_product.shape[0]
        # Process other products' features
        aggregated_other_features = self.other_product_features_layers(x_other_products)

        
        combined_features =  torch.cat((x_user.unsqueeze(1).expand(-1, M, -1),
                                        x_product.unsqueeze(0).expand(N, -1, -1),
                                        aggregated_other_features.unsqueeze(0).expand(N, -1, -1),
                                        prices.view(1, -1, 1).expand(N, -1, -1)),
                                        dim=2)
   

        # Compute utility for each combined feature set
        utilities = self.network(combined_features).squeeze(-1)

        # Incorporate the outside option with utility 0
        zero_utilities = torch.zeros(N, 1, device=utilities.device)
        utilities_with_outside = torch.cat((zero_utilities, utilities), dim=1)

        return utilities_with_outside
        
   

In [43]:
pdlmodel = PDLModel(user_feature_dim=USER_Cont_FEATURES+USER_Dicr_FEATURES,
                       product_feature_dim=Product_Cont_FEATURES+Product_Dicr_FEATURES).to(device)

In [44]:

optimizer = torch.optim.Adam(pdlmodel.parameters(), lr=0.01)

best_val_loss = float('inf')
patience = 15
patience_counter = 0

for epoch in range(1000):
    pdlmodel.train()  
    optimizer.zero_grad()

    outputs = pdlmodel(user_features, product_features, all_x_other_products,prices)
    choice_probabilities = F.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]),decision_train1+1])


    loss.backward()
    optimizer.step()

    # Validation phase
    pdlmodel.eval()  # Set model to evaluation mode


    with torch.no_grad():
        val_outputs = pdlmodel(X_user_val,  product_features, all_x_other_products,prices)
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]),decision_val+1])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        # torch.save(pdlmodel.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break


Epoch 1, Training Loss: 2.379567861557007, Validation Loss: 2.373211622238159
Epoch 2, Training Loss: 2.3734123706817627, Validation Loss: 2.3674166202545166
Epoch 3, Training Loss: 2.367734432220459, Validation Loss: 2.3608593940734863
Epoch 4, Training Loss: 2.361192226409912, Validation Loss: 2.3526690006256104
Epoch 5, Training Loss: 2.353121757507324, Validation Loss: 2.342992067337036
Epoch 6, Training Loss: 2.3436036109924316, Validation Loss: 2.3311798572540283
Epoch 7, Training Loss: 2.3320746421813965, Validation Loss: 2.316425323486328
Epoch 8, Training Loss: 2.317845582962036, Validation Loss: 2.298616409301758
Epoch 9, Training Loss: 2.300743818283081, Validation Loss: 2.2770962715148926
Epoch 10, Training Loss: 2.2800607681274414, Validation Loss: 2.251371145248413
Epoch 11, Training Loss: 2.255384683609009, Validation Loss: 2.2240612506866455
Epoch 12, Training Loss: 2.2291648387908936, Validation Loss: 2.1943764686584473
Epoch 13, Training Loss: 2.200651168823242, Valid

In [45]:
import torch
import torch.nn.functional as F

def calculate_expected_revenue(model,user_features, product_features, all_x_other_products,prices):
    # Ensure model is in evaluation mode
    model.eval()

    with torch.no_grad():  # Disable gradient calculation
        utilities = model(user_features, product_features, all_x_other_products,prices)
        probabilities = F.softmax(utilities, dim=1)  # Softmax over products only

        # Calculate expected revenue for each product
        price_with_outside = torch.cat((torch.zeros(1, device=prices.device),prices), dim=0)
        total_expected_revenue = (probabilities.sum(dim=0)* price_with_outside.unsqueeze(0)).sum()


    return total_expected_revenue.item()  # Convert to Python float

In [46]:
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)
control_prepared_data = prepare_data(X_user_test, X_product,  price)
user_features, product_features, prices, all_x_other_products = control_prepared_data
# Calculate expected revenue
expected_revenue_all_control = calculate_expected_revenue(pdlmodel, user_features, product_features, all_x_other_products, prices)
print(f"Expected Revenue all Control: ${expected_revenue_all_control:.2f}")
all_treated_price = price*discount
treated_prepared_data = prepare_data(X_user_test, X_product,  all_treated_price)
user_features, product_features, prices, all_x_other_products = treated_prepared_data
expected_revenue_all_treated = calculate_expected_revenue(pdlmodel, user_features, product_features, all_x_other_products, prices)
print(f"Expected Revenue all treated: ${expected_revenue_all_treated:.2f}")

Expected Revenue all Control: $3357.73
Expected Revenue all treated: $680.15


In [47]:
pdl = (expected_revenue_all_treated-expected_revenue_all_control)*2

In [48]:
print(f"Absolute Percentage Estimation Error of PDL:  {100*np.abs(pdl-revenue_difference)/revenue_difference:.2f}%")

Absolute Percentage Estimation Error of PDL:  -19.24%


# use dml

In [49]:
class UtilityEstimator(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(UtilityEstimator, self).__init__()
        
        # Layers to process other products' features (z-j)
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim*(NUM_Product-1), NUM_Product),
            nn.ReLU(),
            nn.Linear(NUM_Product, product_feature_dim)
        )

        self.theta0 = nn.Sequential(
            nn.Linear(user_feature_dim + 2 * product_feature_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )
        # Output layer for Theta1 (takes xi, zj, z-j)
        self.theta1 = nn.Sequential(
            nn.Linear(user_feature_dim + 2 * product_feature_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )
        
    def forward(self, x_user, x_product, x_other_products,price):
        N = x_user.shape[0]
        M = x_product.shape[0]
        # Process other products' features
        aggregated_other_features = self.other_product_features_layers(x_other_products)
    

        # Combine features for Theta0
        
        combined_features_theta =  torch.cat((x_user.unsqueeze(1).expand(-1, M, -1),
                                               x_product.unsqueeze(0).expand(N, -1, -1),
                                               aggregated_other_features.unsqueeze(0).expand(N, -1, -1)),
                                                 dim=2)
        theta0_output = self.theta0(combined_features_theta).squeeze(-1)
        theta1_output = self.theta1(combined_features_theta).squeeze(-1)
        
        price = price.unsqueeze(-1)  
        utility = theta0_output + theta1_output * price.squeeze(-1)

        # Include the outside option (utility = 0)
        zero_utilities = torch.zeros(x_user.shape[0], 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)
        
        return utilities_with_outside,theta0_output,theta1_output


In [50]:
dml_model = UtilityEstimator(user_feature_dim=USER_Cont_FEATURES+USER_Dicr_FEATURES,
                       product_feature_dim=Product_Cont_FEATURES+Product_Dicr_FEATURES).to(device)

In [51]:
X_user_train1, X_user_val, decision_train1,decision_val = train_test_split(X_user_train,decision_train,test_size=0.1,random_state=34)

In [52]:
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        all_x_other_products.append(other_products)

    # Convert lists to tensor
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
  

    return user_features, product_features, prices, all_x_other_products


In [53]:
price = price.to(device)
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount) * prod_randomization))
user_features, product_features, prices, all_x_other_products = prepared_data
user_features.shape, product_features.shape, prices.shape, all_x_other_products.shape

(torch.Size([4500, 5]),
 torch.Size([10, 5]),
 torch.Size([10]),
 torch.Size([10, 45]))

In [54]:
import torch.nn.functional as F
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)

best_val_loss = float('inf')
patience = 15
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()
    
    outputs = dml_model(user_features, product_features, all_x_other_products,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), decision_train1+1 ])

    loss.backward()
    optimizer.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val,  product_features, all_x_other_products,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]),decision_val+1])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break


Epoch 1, Training Loss: 2.397836923599243, Validation Loss: 2.379599094390869
Epoch 2, Training Loss: 2.3763084411621094, Validation Loss: 2.3584401607513428
Epoch 3, Training Loss: 2.3553483486175537, Validation Loss: 2.3401260375976562
Epoch 4, Training Loss: 2.337233781814575, Validation Loss: 2.3230464458465576
Epoch 5, Training Loss: 2.320643186569214, Validation Loss: 2.3053343296051025
Epoch 6, Training Loss: 2.303337335586548, Validation Loss: 2.285661220550537
Epoch 7, Training Loss: 2.2839224338531494, Validation Loss: 2.2632639408111572
Epoch 8, Training Loss: 2.262124538421631, Validation Loss: 2.2373857498168945
Epoch 9, Training Loss: 2.237248659133911, Validation Loss: 2.2089905738830566
Epoch 10, Training Loss: 2.2103540897369385, Validation Loss: 2.1780762672424316
Epoch 11, Training Loss: 2.1808104515075684, Validation Loss: 2.143239736557007
Epoch 12, Training Loss: 2.147589683532715, Validation Loss: 2.1067798137664795
Epoch 13, Training Loss: 2.1134324073791504, Va

In [55]:
import torch
import torch.nn.functional as F

def calculate_expected_revenue(model,user_features, product_features, all_x_other_products,prices):
    # Ensure model is in evaluation mode
    model.eval()

    with torch.no_grad():  # Disable gradient calculation
        utilities = model(user_features, product_features, all_x_other_products,prices)[0]
        probabilities = F.softmax(utilities, dim=1)  # Softmax over products only

        # Calculate expected revenue for each product
        price_with_outside = torch.cat((torch.zeros(1, device=prices.device),prices), dim=0)
        total_expected_revenue = (probabilities.sum(dim=0)* price_with_outside.unsqueeze(0)).sum()


    return total_expected_revenue.item()  # Convert to Python float

In [56]:
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)
control_prepared_data = prepare_data(X_user_test, X_product,  price)
user_features, product_features, prices, all_x_other_products = control_prepared_data
# Calculate expected revenue
expected_revenue_all_control = calculate_expected_revenue(dml_model, user_features, product_features, all_x_other_products, prices)
print(f"Expected Revenue all Control: ${expected_revenue_all_control:.2f}")
all_treated_price = price*discount
treated_prepared_data = prepare_data(X_user_test, X_product,  all_treated_price)
user_features, product_features, prices, all_x_other_products = treated_prepared_data
expected_revenue_all_treated = calculate_expected_revenue(dml_model, user_features, product_features, all_x_other_products, prices)
print(f"Expected Revenue all treated: ${expected_revenue_all_treated:.2f}")

Expected Revenue all Control: $3387.49
Expected Revenue all treated: $663.02


In [57]:
expected_revenue_all_treated-expected_revenue_all_control

-2724.4629516601562

# debias the GTE estimator:

In [58]:
test_prepared_data = prepare_data(X_user_test, X_product,  price*(discount*prod_randomization))
user_features, product_features, prices, all_x_other_products = test_prepared_data

# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products,prices)


In [59]:
theta1_output.shape

torch.Size([5000, 10])

# use formulation debias for H_i

In [60]:
def H_theta(theta0_output,theta1_output,all_treated_price,price):
    N = theta0_output.shape[0]
    M = NUM_Product
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = all_treated_price.unsqueeze(0).expand(N, M)
    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price


    # Include the outside option (utility = 0)
    zero_utilities = torch.zeros(N, 1, device=all_treated_uti.device)
    all_treated_uti = torch.cat((zero_utilities,all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities,all_control_uti), dim=1)

    all_treated_probabilities = F.softmax(all_treated_uti, dim=1)
    all_control_probabilities = F.softmax(all_control_uti, dim=1)

    price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
    treated_price_with_outside =  torch.cat((torch.zeros(1, device=all_treated_price.device),all_treated_price), dim=0)

    H = torch.sum(all_treated_probabilities*treated_price_with_outside - all_control_probabilities*price_with_outside,dim=1)
    expsum_treated = torch.sum(torch.exp(all_treated_uti),dim=1)
    expsum_control = torch.sum(torch.exp(all_control_uti),dim=1)

    expsum_treated_expanded = expsum_treated.unsqueeze(1).expand(-1, all_treated_uti.shape[1])  # Shape [N, M+1]
    expsum_control_expanded = expsum_control.unsqueeze(1).expand(-1, all_control_uti.shape[1])  # Shape [N, M+1]

    H_theta0 = torch.sum((torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded-\
                          torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded)\
                         *price_with_outside,dim=1)
    H_theta1 = torch.sum(price_with_outside*(torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded*treated_price_with_outside-\
                                             torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded*price_with_outside),dim=1)


    return H,H_theta0,H_theta1


In [61]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

In [62]:
def l_theta(theta0_output,theta1_output,adjusted_price,decision_test):
    N = theta0_output.shape[0]
    M = NUM_Product
    expand_adjusted_price = adjusted_price.unsqueeze(0).expand(N, M)
    uti = theta0_output + theta1_output * expand_adjusted_price
    adjusted_price_with_outside =  torch.cat([torch.zeros(1, device=adjusted_price.device),adjusted_price])

    # Include the outside option (utility = 0)
    zero_utilities = torch.zeros(N, 1, device=uti.device)
    uti = torch.cat((zero_utilities,uti), dim=1)

    probabilities = F.softmax(uti, dim=1)
    prod_indices = torch.ones(NUM_Product, device=device)
    prod_indices = torch.cat([torch.zeros(1,device=device),prod_indices])
    ltheta0 = probabilities[torch.arange(decision_test.size(0)), decision_test+1] -prod_indices[decision_test+1]
    ltheta1 = (probabilities[torch.arange(decision_test.size(0)), decision_test+1] * adjusted_price_with_outside[decision_test+1]) - adjusted_price_with_outside[decision_test+1]


    return ltheta0,ltheta1

In [63]:
price = price.to(device)

In [64]:
adjusted_price = price*(discount*prod_randomization).to(device)
decision_test = decision_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output,theta1_output,adjusted_price,decision_test)

In [ ]:
import torch
import torch.nn.functional as F

def lambdainv(theta0_output, theta1_output, price, decision_test,epsilon =10):
    N = theta0_output.shape[0]
    M = NUM_Product
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = discount*price.unsqueeze(0).expand(N, M)

    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price

    # Include the outside option (utility = 0)
    zero_utilities = torch.zeros(N, 1, device=all_control_uti.device)
    all_treated_uti = torch.cat((zero_utilities,all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities,all_control_uti), dim=1)

    # Calculate probabilities using softmax
    probabilities_control = F.softmax(all_control_uti, dim=1)
    probabilities_treated = F.softmax(all_treated_uti, dim=1)

    # Extract probabilities of chosen products
    chosen_prob_control = probabilities_control[torch.arange(N), decision_test]
    chosen_prob_treated = probabilities_treated[torch.arange(N), decision_test]

    # Calculate second derivatives
    ltheta00 = chosen_prob_control * (1 - chosen_prob_control) + chosen_prob_treated * (1 - chosen_prob_treated)
    ltheta01 = chosen_prob_control * (1 - chosen_prob_control) * expand_price[torch.arange(N), decision_test] + \
            chosen_prob_treated * (1 - chosen_prob_treated) * (discount * expand_price[torch.arange(N), decision_test])
    ltheta11 = chosen_prob_control * (1 - chosen_prob_control) * expand_price[torch.arange(N), decision_test]**2 + \
            chosen_prob_treated * (1 - chosen_prob_treated) * (discount * expand_price[torch.arange(N), decision_test])**2
    ltheta00=ltheta00/2
    ltheta01=ltheta01/2
    ltheta11=ltheta11/2

    # Form the 2x2 Hessian matrices for each instance
    ltheta00 = ltheta00.unsqueeze(1).unsqueeze(2)
    ltheta01 = ltheta01.unsqueeze(1).unsqueeze(2)
    ltheta11 = ltheta11.unsqueeze(1).unsqueeze(2)

    top_row = torch.cat((ltheta00, ltheta01), dim=2)
    bottom_row = torch.cat((ltheta01, ltheta11), dim=2)

    L_matrix = torch.cat((top_row, bottom_row), dim=1)

    # Regularization and inversion
    
    identity_matrix = torch.eye(2, dtype=L_matrix.dtype, device=L_matrix.device) * epsilon
    L_matrix_reg = L_matrix + identity_matrix.unsqueeze(0).unsqueeze(0)
    L_inv = torch.linalg.inv(L_matrix_reg)

    return L_inv


In [66]:
epsilon_list = [0.001,0.01,0.1,0.5,1,5,10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, decision_test, epsilon).float()
    
        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()  
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()  
    
        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0)) 
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)  
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0
    
        # Calculate sdl and dedl
        sdl = H.sum().cpu().detach().numpy() * 2
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 2
    
        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)
    
        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

In [67]:
sdl = H.sum().cpu().detach().numpy()*2

In [68]:
dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy())*2

In [69]:
sdl,dedl,best_epsilon

(-5448.92626953125, -4466.58251953125, 1)

In [70]:
print(f"Absolute Percentage Estimation Error of SDL:  {100*np.abs(sdl-revenue_difference)/revenue_difference:.2f}%")
print(f"Absolute Percentage Estimation Error of SP MNL:  {100*np.abs(dedl-revenue_difference)/revenue_difference:.2f}%")

Absolute Percentage Estimation Error of SDL:  -21.33%
Absolute Percentage Estimation Error of SP MNL:  -0.55%


In [71]:
naive_pe = (naive - true) / true
linear_pe = (linear - true) / true
pdl_pe = (pdl - true) / true
sdl_pe = (sdl - true) / true
dedl_pe = (dedl - true) / true
naive_mse = (naive - true)**2
linear_mse =(linear - true)**2
pdl_mse = (pdl - true)**2
sdl_mse = (sdl - true)**2
dedl_mse = (dedl - true)**2
naive_e = (naive - true)
linear_e =(linear - true)
pdl_e = (pdl - true)
sdl_e = (sdl - true)
dedl_e = (dedl - true)

In [72]:
print(naive_pe,linear_pe,pdl_pe,sdl_pe,dedl_pe,naive_e,linear_e,pdl_e,sdl_e,dedl_e,naive_mse,linear_mse,pdl_mse,sdl_mse,dedl_mse)

1.1776921083296106 0.23181642538978905 0.1924034075705658 0.21328039807193516 -0.005452313124395358 -5289.09679633379 -1041.103616297245 -864.09702450037 -957.857034265995 24.486715734004974 27974544.92098836 1083896.7398672013 746663.667750393 917490.0980928476 599.5992474379668
